In [59]:
import torch
import torch.multiprocessing as mp
import os
from torch.utils.data import DataLoader, TensorDataset
from tools.average_embeddings import average_embeddings
from tools.train_tree import train_tree
from math import log2
from DecisionTree import DecisionTree
import random

print(os.getcwd())

/Users/yk/Desktop/comp9444/proj/COVID-Sentiment/models


In [60]:
data = average_embeddings("../features_extraction/word2vec.pt")

In [61]:
data_size = len(data)
# print(data_size)
# train = data[:int(data_size * 0.8)]
# test = data[int(data_size * 0.8):]
train = []
c1 = c2 = c3 = 0
for i in data:
    label = i[1]
    if label == -1 and c1 <= 10:
        train.append(i)
        c1 += 1
    elif label == 0 and c2 <= 10:
        train.append(i)
        c2 += 1
    elif label == 1 and c3 <= 10:
        train.append(i)
        c3 += 1
        
    if c1 == 10 and c2 == 10 and c3 == 10:
        break

random.shuffle(train)

test = data[:-20]

In [62]:
X_train = torch.tensor([i[0] for i in train])
y_train = torch.tensor([i[1] for i in train])


X_test = torch.tensor([i[0] for i in test])
y_test = torch.tensor([i[1] for i in test])

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=True)

In [63]:
class RandomForest:
    def __init__(self, n_trees=5, max_depth=3, min_samples_split=2):
        self.n_trees = n_trees
        self.trees = [DecisionTree(max_depth=max_depth, min_samples_split=min_samples_split) for _ in range(n_trees)]
        
    def fit(self, X, y):
        # for tree in self.trees:
        #     # Select random samples
        #     indices = torch.randint(0, len(X), (len(X),))
        #     X_samples, y_samples = X[indices], y[indices]
        #     tree.fit(X_samples, y_samples)
        
        with mp.Pool() as pool:
            self.trees = pool.starmap(train_tree, [(tree, X, y) for tree in self.trees])
            
            
    
    def predict(self, X):
        predictions = torch.stack([tree.predict(X) for tree in self.trees])
        return torch.mode(predictions).values
            

In [69]:
data = list(data)
random.shuffle(data)

X_all = torch.stack([torch.tensor(i[0]) for i in data])
y_all = torch.tensor([i[1] for i in data])

n_samples = len(data)
folds = n_samples // 5
indices = list(range(n_samples))

accuracies = []

for fold in range(5):
    print(f"\nFold {fold + 1}")
    val_start = fold * folds
    val_end = (fold + 1) * folds if fold != 4 else n_samples
    val_idx = indices[val_start:val_end]
    train_idx = indices[:val_start] + indices[val_end:]
    X_train = X_all[train_idx]
    y_train = y_all[train_idx]
    X_val = X_all[val_idx]
    y_val = y_all[val_idx]

    model = RandomForest(n_trees=5, max_depth=3, min_samples_split=2)
    model.fit(X_train, y_train)

    preds = []
    for i in X_val:
        preds.append(model.predict(i))
        
    preds = torch.tensor(preds)
    acc = (preds == y_val).float().mean().item()
    print(f"Accuracy: {acc:.4f}")
    accuracies.append(acc)

print(f"\nAverage Accuracy over 5 folds: {sum(accuracies) / len(accuracies):.4f}")


Fold 1
Accuracy: 0.8500

Fold 2
Accuracy: 0.8500

Fold 3
Accuracy: 0.7500

Fold 4
